## ライブラリ rCharts を使った javascript グラフ作成

ライブラリ rCharts を使うと、R から比較的容易に javascript のグラフを作成することが可能になる。

In [1]:
library(rCharts)
library(ltm)

Loading required package: MASS
Loading required package: msm
Loading required package: polycor
Loading required package: mvtnorm
Loading required package: sfsmisc


In [4]:
csvdata <- read.csv('~/IrtWeb/irt_sample.csv', header = TRUE)

In [7]:
result <- ltm(csvdata~z1, IRT.param = TRUE)

ltm パッケージを使って、IRT 分析を行い、その結果から種々のパラメータを抜き出す。

* 項目数： valsize  
* 項目名： item_names  
* 項目ごとの識別度 : a  
* 項目ごとの困難度 : b  

In [36]:
valsize <- dim(csvdata)[2]

In [34]:
a <- result$coefficient[,2]
b <- summary(result)$coefficients[,1][1:valsize]
item_names <- rownames(result$coefficient)

print(item_names)
print(a)
print(b)

[1] "A" "B" "C" "D" "E"
        A         B         C         D         E 
 1.215814  1.574269 16.899394  1.169672  1.181162 
   Dffclt.A    Dffclt.B    Dffclt.C    Dffclt.D    Dffclt.E 
-1.17618278 -0.39160134 -0.05290487  0.20131818  1.15882655 


IRT における 2PL モデルに基づく確率関数を設定

In [13]:
model2pl <- function(x, a, b) {
    return (1.0 / (1.0 + exp(-1.701 * a * (x - b))))
}

ICCを描くために、項目ごとに(x, y) のデータを作成する。x の値として、 -4 から 4 までの範囲で 0.1 刻みのデータを xdata として用意する。

項目それぞれに対して、xdata の値における上記モデルの p の値を求める。
グラフ作成関数の入力として使えるようにデータ構造を設定する。
項目ごとに（項目名, x値, y値）を作成し、列方向にすべての項目をつなげて作る。

In [40]:
data <- data.frame()
xdata <- seq(-4.0, 4.01, by=0.1)
for (i in 1:valsize) {
    ydata_tmp <- model2pl(xdata, a[i], b[i])
    data_tmp <- data.frame(type=rep(item_names[i], length(xdata)), x=xdata, y=ydata_tmp)
    data <- rbind(data, data_tmp)
}

Javascript グラフ作成パッケージとしてはいくつかの選択肢があり、その中で HighCharts を駆動する例を示す。
hPlot() 関数でグラフオブジェクトを作成し、その属性を各種設定していく。
$show() 関数で画面に表示されるが、$print() 関数を起動すると生成された HTML/Javascript が示される。

In [48]:
p <- hPlot(y~x, group="type", data=data, type = 'line', title = '項目特性曲線', subtitle = 'Item Characteristic Curves')
p$chart(zoomType = "xy")
p$exporting(enabled = T)
p$xAxis(title = list(text = "Ability"), min = -4.0, max = 4.0, gridLineWidth = 1)
p$yAxis(title = list(text = "Probability"), min = 0.0, max = 1.0, gridLineWidth = 1)
p$plotOptions(line = list(marker = list(enabled = F)))
p$legend(align = 'right', verticalAlign = 'bottom', layout = 'vertical')
p$show()